In [1]:
import csv

csvfile = open("decode.csv")
insts = csv.DictReader(csvfile)
insts = list(filter(lambda x: x["pat"] != "", insts))
for i in insts:
    i["name"] = i["name"].strip()
# reg_we
# op1_sel
# op2_sel
# op1_U
# op2_U
# alu_sel
# wb_sel
# invalid
# break

In [2]:
def process_fill_zero(inst):
    opt_fields = ["reg_we", "op1_U", "op2_U"]
    for field in opt_fields:
        inst[field] = int(inst[field] == "1")
    inst["invalid"] = "0"
    inst["break"] = int(inst["name"] == "ebreak")
    # return inst

In [3]:
def process_op_sel(inst):
    op1_sel_dict = {"rs1": "00", "pc": "01", "zero": "10", "": "??"}
    op2_sel_dict = {"rs2": "0", "imm": "1", "": "?"}
    inst["op1_sel"] = op1_sel_dict[inst["op1_sel"]]
    inst["op2_sel"] = op2_sel_dict[inst["op2_sel"]]
    return inst

In [4]:
def process_alu_sel(inst):
    alu_sel = [
        "add",
        "sub",
        "left",
        "right",
        "arithmetic",
        "lt",
        "and",
        "or",
        "xor",
        "mul",
        "mulh",
        "div",
        "rem",
    ]
    alu_sel_dict = {"": "????"}
    for i in range(len(alu_sel)):
        alu_sel_dict[alu_sel[i]] = str(bin(i))[2:].zfill(4)
    inst["alu_sel"] = alu_sel_dict[inst["alu_sel"]]
    # return inst

In [5]:
def process_wb_sel(inst):
    wb_sel_dict = {"alu": "00", "mem": "01", "snpc": "10", "": "??"}
    inst["wb_sel"] = wb_sel_dict[inst["wb_sel"]]
    # return inst

In [6]:
def process_branch_jump_cmp(inst):
    inst["is_beq"] = int(inst["name"] == "beq")
    inst["is_bne"] = int(inst["name"] == "bne")
    inst["is_blt"] = int(inst["name"] == "blt" or inst["name"] == "bltu")
    inst["is_bge"] = int(inst["name"] == "bge" or inst["name"] == "bgeu")
    inst["cmp_U"] = int(inst["name"] in ["bltu", "bgeu", "sltu", "sltiu"])

    inst["is_jump"] = int((inst["name"] == "jal") or (inst["name"] == "jalr"))
    # return inst

In [7]:
def process_load_unsigned(inst):
    load_unsigned = ["lbu", "lhu"]
    inst["load_U"] = int(inst["name"] in load_unsigned)
    # return inst

In [8]:
def process_mem(inst):
    load = ["lb", "lbu", "lh", "lhu", "lw"]
    store = ["sb", "sh", "sw"]
    len_dict = {"1": "00", "2": "01", "4": "10", "": "??"}
    inst["mr"] = int(inst["name"] in load)
    inst["mw"] = int(inst["name"] in store)
    inst["len"] = len_dict[inst["len"]]
    # return inst

In [9]:
def process_inst_type(inst):
    types_dict = {
        "I": "000",
        "U": "001",
        "S": "010",
        "B": "011",
        "J": "100",
        "R": "???",
    }
    inst["inst_type"] = types_dict[inst["inst_type"]]
    # return inst

In [10]:
def filter_branch(inst: dict):
    return inst

In [11]:
funcs = [
    # reg_we op1_U op2_U invalid break
    process_fill_zero,
    # op1_sel op2_sel
    process_op_sel,
    # alu_sel
    process_alu_sel,
    # wb_sel
    process_wb_sel,
    # is_beq is_bne is_blt is_bge is_jump
    process_branch_jump_cmp,
    # load_U
    process_load_unsigned,
    # mr mw
    process_mem,
    # inst_type
    process_inst_type,
    #
    # filter_branch,
]
for i in insts:
    for f in funcs:
        f(i)
    for k, v in i.items():
        i[k] = str(v)
# insts = processed

In [12]:
for i in insts:
    print(i)

{'pat': '??????? ????? ????? ??? ????? 01101 11', 'name': 'lui', 'reg_we': '1', 'op1_sel': '10', 'op2_sel': '1', 'op1_U': '0', 'op2_U': '0', 'alu_sel': '0000', 'wb_sel': '00', 'inst_type': '001', 'len': '??', 'invalid': '0', 'break': '0', 'is_beq': '0', 'is_bne': '0', 'is_blt': '0', 'is_bge': '0', 'cmp_U': '0', 'is_jump': '0', 'load_U': '0', 'mr': '0', 'mw': '0'}
{'pat': '??????? ????? ????? ??? ????? 00101 11', 'name': 'auipc', 'reg_we': '1', 'op1_sel': '01', 'op2_sel': '1', 'op1_U': '0', 'op2_U': '0', 'alu_sel': '0000', 'wb_sel': '00', 'inst_type': '001', 'len': '??', 'invalid': '0', 'break': '0', 'is_beq': '0', 'is_bne': '0', 'is_blt': '0', 'is_bge': '0', 'cmp_U': '0', 'is_jump': '0', 'load_U': '0', 'mr': '0', 'mw': '0'}
{'pat': '??????? ????? ????? ??? ????? 11011 11', 'name': 'jal', 'reg_we': '1', 'op1_sel': '01', 'op2_sel': '1', 'op1_U': '0', 'op2_U': '0', 'alu_sel': '0000', 'wb_sel': '10', 'inst_type': '100', 'len': '??', 'invalid': '0', 'break': '0', 'is_beq': '0', 'is_bne': '0

In [13]:
class Signal:
    l: int = 0
    r: int = 0
    name: str = ""


offset = 0
for k, v in insts[0].items():
    signal = Signal()
    if k != "pat" and k != "name":
        signal.name = k
        signal.l = offset
        signal.r = offset + len(v) - 1
        offset += len(v)
        print(f"{signal.name:10}({signal.l:2},{signal.r:2})")

reg_we    ( 0, 0)
op1_sel   ( 1, 2)
op2_sel   ( 3, 3)
op1_U     ( 4, 4)
op2_U     ( 5, 5)
alu_sel   ( 6, 9)
wb_sel    (10,11)
inst_type (12,14)
len       (15,16)
invalid   (17,17)
break     (18,18)
is_beq    (19,19)
is_bne    (20,20)
is_blt    (21,21)
is_bge    (22,22)
cmp_U     (23,23)
is_jump   (24,24)
load_U    (25,25)
mr        (26,26)
mw        (27,27)


In [14]:
offset = 0
signals = {}
for k, v in insts[0].items():
    if k == "pat" or k == "name":
        continue
    signal = Signal()
    signals[k] = signal
    signal.name = k
    signal.l = offset
    signal.r = offset + len(v) - 1
    offset += len(v)
    print(f"val {k} = UInt({len(v)}.W) // ({signal.l},{signal.r})")
control_len = offset

val reg_we = UInt(1.W) // (0,0)
val op1_sel = UInt(2.W) // (1,2)
val op2_sel = UInt(1.W) // (3,3)
val op1_U = UInt(1.W) // (4,4)
val op2_U = UInt(1.W) // (5,5)
val alu_sel = UInt(4.W) // (6,9)
val wb_sel = UInt(2.W) // (10,11)
val inst_type = UInt(3.W) // (12,14)
val len = UInt(2.W) // (15,16)
val invalid = UInt(1.W) // (17,17)
val break = UInt(1.W) // (18,18)
val is_beq = UInt(1.W) // (19,19)
val is_bne = UInt(1.W) // (20,20)
val is_blt = UInt(1.W) // (21,21)
val is_bge = UInt(1.W) // (22,22)
val cmp_U = UInt(1.W) // (23,23)
val is_jump = UInt(1.W) // (24,24)
val load_U = UInt(1.W) // (25,25)
val mr = UInt(1.W) // (26,26)
val mw = UInt(1.W) // (27,27)


In [15]:
isa = {}
for i in insts:
    Value = str()
    for k, v in i.items():
        if k == "name" or k == "pat":
            continue
        Value += v
    print(f'BitPat("b{i["pat"]}") -> BitPat("b{Value}"),')
    i["control"] = Value
    isa[i["name"]] = {"pat": i["pat"], "ctl": Value}

BitPat("b??????? ????? ????? ??? ????? 01101 11") -> BitPat("b110100000000001??00000000000"),
BitPat("b??????? ????? ????? ??? ????? 00101 11") -> BitPat("b101100000000001??00000000000"),
BitPat("b??????? ????? ????? ??? ????? 11011 11") -> BitPat("b101100000010100??00000001000"),
BitPat("b??????? ????? ????? ??? ????? 11001 11") -> BitPat("b100100000010000??00000001000"),
BitPat("b??????? ????? ????? 000 ????? 11000 11") -> BitPat("b0011000000??011??00100000000"),
BitPat("b??????? ????? ????? 001 ????? 11000 11") -> BitPat("b0011000000??011??00010000000"),
BitPat("b??????? ????? ????? 100 ????? 11000 11") -> BitPat("b0011000000??011??00001000000"),
BitPat("b??????? ????? ????? 101 ????? 11000 11") -> BitPat("b0011000000??011??00000100000"),
BitPat("b??????? ????? ????? 110 ????? 11000 11") -> BitPat("b0011000000??011??00001010000"),
BitPat("b??????? ????? ????? 111 ????? 11000 11") -> BitPat("b0011000000??011??00000110000"),
BitPat("b??????? ????? ????? 000 ????? 00000 11") -> BitPat(

In [16]:
invalid = ["?" for i in range(control_len)]
invalid[signals["invalid"].l] = "1"
invalid = "".join(invalid)
isa["invalid"] = {"pat": "", "ctl": invalid}

In [17]:
print(isa["addi"]["pat"], isa["addi"]["ctl"])
print(isa["invalid"]["pat"], isa["invalid"]["ctl"])

??????? ????? ????? 000 ????? 00100 11 100100000000000??00000000000
 ?????????????????1??????????
